In [ ]:
# THIS IS MY MODEL, ALSO CALLED DELGADOVENEZIAN

def solve_1D_BPP_WB(cargo, deferred_items=None, attempted_combinations=None, number_of_opened_uld=None, packed_ULDs=None, open_new_uld=False, open_new_extra_uld=False, deferred_items_optimal_flag=False):
    """
    Solve 1D Bin Packing Problem combined with Weight and Balance
    """
    m = Model("1D_BPP_WB")

    '''Parameters'''
    
    loadfactor = 0.8
    a_lat_TOW = 0.5
    b_lat_TOW = 0.5
    a_lat_LW = 0.5
    b_lat_LW = 0.5

    '''Open a new extra ULD compared to the actual set of ULDs [Feedback loop]'''

    if open_new_extra_uld is True:
        max_index = max([j.index for j in cargo.uld])
        extra_PMC = ULD(max_index + 1, 'PMC', 'PMC' + '-' + str(max_index + 1))
        extra_AKE = ULD(max_index + 2, 'AKE', 'AKE' + '-' + str(max_index + 2))
        cargo.uld.append(extra_PMC)
        cargo.uld.append(extra_AKE)
        cargo.define_parameters_ULD()

    '''Decision variables'''

    f = {}
    u = {}
    p = {}
    z = {}

    for j in cargo.uld:
        for t in aircraft.loadlocations:
            f[j.index, t.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'f_{j.index}_{t.index}')

    for j in cargo.uld:
        u[j.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'u_{j.index}')
    
    #aca solo defino p para los uld que pueden tener items, me ahorro una restriccion y muchas variables
    for i in cargo.items:
        for j in cargo.uld:
            p[i.index, j.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'p_{i.index}_{j.index}')
    for i in cargo.items:
        for j in cargo.uld:
            for t in aircraft.loadlocations:
                z[i.index, j.index, t.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'z_{i.index}_{j.index}_{t.index}')
    
    '''Feedback loop actions'''

    for uld, items in packed_ULDs.items():
        for j in cargo.uld:
            if j.isNeitherBAXnorBUPnorT and str(uld) == str(j.serialnumber):
                u[j.index].lb = 1
                for i in cargo.items:
                    p[i.index, j.index].lb = 0
                    p[i.index, j.index].ub = 0
                for item in items:
                    for i in cargo.items:
                        if str(i.serialnumber) == str(item):
                            p[i.index, j.index].lb = 1
                            p[i.index, j.index].ub = 1

    if deferred_items is not None and open_new_uld == False:
        for j, items in deferred_items.items():
            for i in items:
                p[i.index, j.index].lb = 0
                p[i.index, j.index].ub = 0

    m.update()

    '''Model Sense'''

    m.ModelSense = GRB.MINIMIZE

    m.update()

    '''Objective function 1 --> Maximize the %MAC [Highest Priority]'''

    ZFW_index_obj = m.addVar(vtype=GRB.CONTINUOUS, name="ZFW_index_obj")

    m.addConstr(ZFW_index_obj == aircraft.DOI + aircraft.define_INDEX_PAX() +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C1 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C2 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C3 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C4)
    
    m.addConstr(aircraft.define_INDEX_ZFW_fwd(aircraft.aircraft_type) <= ZFW_index_obj, "ZFW_index_fwd_constraint")
    m.addConstr(ZFW_index_obj <= aircraft.define_INDEX_ZFW_aft(aircraft.aircraft_type), "ZFW_index_aft_constraint")

    #obj 1
    MAC_obj = (((aircraft.C * (ZFW_index_obj - aircraft.K)) / aircraft.ZFW) + aircraft.reference_arm - aircraft.lemac) / (aircraft.mac_formula / 100)

    m.setObjectiveN(MAC_obj, index=0, priority=6, weight=-1)

    m.update()

    #eliminar obj 2
    # '''Objective function 2 --> Place larger volume item in PMC/PAG ULDs and small volume item in AKE ULDs [Medium Priority]'''
    # volume_threshold = data_analysis.threshold_volume_in_AKE() * 1000000

    # score_AKE = {}
    # score_PMC_PAG = {}

    # for j in cargo.uld:
    #     if j.isNeitherBAXnorBUPnorT:
    #         score_AKE[j.index] = m.addVar(vtype=GRB.CONTINUOUS, name=f'score_AKE_{j.index}', lb=0)
    #         score_PMC_PAG[j.index] = m.addVar(vtype=GRB.CONTINUOUS, name=f'score_PMC_PAG_{j.index}', lb=0)

    #         m.addConstr(score_AKE[j.index] == quicksum(p[i.index, j.index] * (volume_threshold - i.volume) for i in cargo.items if i.volume < volume_threshold and 'AKE' in j.type))
    #         m.addConstr(score_PMC_PAG[j.index] == quicksum(p[i.index, j.index] * (volume_threshold - i.volume) for i in cargo.items if i.volume < volume_threshold and ('PMC' in j.type or 'PAG' in j.type)))
    
    
    # #obj 2
    # obj_volume_preference = quicksum(score_AKE[j.index] + score_PMC_PAG[j.index] for j in cargo.uld if j.isNeitherBAXnorBUPnorT)
    # m.setObjectiveN(obj_volume_preference, index=1, priority=5, weight=1)
    # m.update()

    '''Objective function 3 --> Minimize number of ULDs opened [Medium Priority]'''

    #obj 3
    obj2 = quicksum(u[j.index] for j in cargo.uld)
    m.setObjectiveN(obj2, index=1, priority=5, weight=1)
    m.update()

    #obj 4
    ## aca iba el obj 4 antiguo pero lo eliminamos, ahora el obj 5 antiguo es el obj 4 actual
    # vamos a agregar el obj 4 del latex
    obj_volume_total = quicksum(i.volume * p[i.index, j.index] for i in cargo.items for j in cargo.uld if j.isNeitherBAXnorBUPnorT)
    m.setObjectiveN(obj_volume_total, index=2, priority=4, weight=-1)

    '''Objective function 4 --> Minimize the separation of items with the same serialnumber prefix over different ULDs [Medium Priority]'''


    booking_groups = cargo.get_prefix_groups()

    # probando mi version de SP, esta es la que hicimos con el profe y no resulta, si pongo esto se separan los items
    # SP = m.addVars(booking_groups.keys(), vtype=GRB.BINARY, name="SP")
    # obj_separation = quicksum(SP[b_i] for b_i in booking_groups.keys())
    
    # opcion 2
    # Create SP variables for ALL ULDs (not just cargo ULDs), esto si funciona
    # SP = {}
    # for b_i, items in booking_groups.items():
    #     for j in cargo.uld:  # ALL ULDs
    #         SP[b_i, j.index] = m.addVar(vtype=GRB.BINARY, name=f'SP_{b_i}_{j.index}')
    
    # obj_separation = quicksum(SP[b_i, j.index] for b_i in booking_groups.keys() for j in cargo.uld)
    # m.setObjectiveN(obj_separation, index=4, priority=2, weight=1)
    # m.update()

    # Define new variables
    Y = {}  # Number of ULDs used for objects of group bi
    Z = {}  # 1 if at least one object of bi is in ULD j
    for b_i in booking_groups.keys():
        Y[b_i] = m.addVar(vtype=GRB.INTEGER, lb=0, name=f'Y_{b_i}')
        for j in cargo.uld:
            if j.isNeitherBAXnorBUPnorT:  # Only for non-BAX/BUP/T ULDs
                Z[b_i, j.index] = m.addVar(vtype=GRB.BINARY, name=f'Z_{b_i}_{j.index}')
    # New objective function: Minimize sum_bi Y_bi
    obj_separation = quicksum(Y[b_i] for b_i in booking_groups.keys())
    m.setObjectiveN(obj_separation, index=3, priority=3, weight=1)
    m.update()

# antigua restr 6
    '''Objective function 5 --> Minimize the proximity score of BAX ULDs [Low Priority]'''
    
    obj4 = quicksum(aircraft.define_proximity_score_loadlocation(t) * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations if j.isBAX)
    m.setObjectiveN(obj4, index=4, priority=2, weight=1)

    m.update()

    '''Constraints'''

    # DV-Extra: Apertura Mínima de ULDs - Minimum ULDs to open (feedback loop)
    m.addConstr(quicksum(u[j.index] for j in cargo.uld if j.isNeitherBAXnorBUPnorT) >= number_of_opened_uld, name='open_new_uld_constraint')

    # DV-Extra2: Uso de ULD - If ULD is opened, at least one item must be in it
    for j in cargo.uld:
        if j.isNeitherBAXnorBUPnorT:
            m.addConstr(quicksum(p[i.index, j.index] for i in cargo.items) >= u[j.index], name=f'C_new_uld_{j.index}')

    # DV1: ULD weight capacity - Total weight in ULD cannot exceed its capacity
    for j in cargo.uld:
        m.addConstr(quicksum(i.weight * p[i.index, j.index] for i in cargo.items) <= j.max_weight * u[j.index], name=f'C1_{j.index}')
    

    # DV2: ULD volume capacity - Total volume in ULD cannot exceed its capacity
    for j in cargo.uld:
        m.addConstr(quicksum(i.volume * p[i.index, j.index] for i in cargo.items) <= j.volume * u[j.index] * loadfactor, name=f'C2_{j.index}')

    # DV2-Note: Minimum volume utilization constraint (commented out)
    # alpha = 0.2 ## despues poner esto en parametros correctos
    # for j in cargo.uld:
    #     if j.isNeitherBAXnorBUPnorT:
    #         m.addConstr(quicksum(i.volume * p[i.index, j.index] for i in cargo.items) >= alpha * j.volume * u[j.index], name=f'C_min_volume_{j.index}')
    
    # DV3: Item assignment - Every item must be placed in exactly one ULD
    for i in cargo.items:
        m.addConstr(quicksum(p[i.index, j.index] for j in cargo.uld if j.isNeitherBAXnorBUPnorT) == 1, name=f'C3_{i.index}')

    # DV4: ULD position assignment - Each active ULD assigned to exactly one position
    for j in cargo.uld:
        m.addConstr(quicksum(f[j.index, t.index] for t in aircraft.loadlocations) == u[j.index], name=f'C_combi_1_{j.index}')

    # DV5: Position occupancy - Each position can hold at most one ULD
    for t in aircraft.loadlocations:
        m.addConstr(quicksum(f[j.index, t.index] for j in cargo.uld) <= 1, name=f'C4_{t.index}')

    # DV6: Compatible positions - ULD type must match position type
    for j in cargo.uld:
        m.addConstr(quicksum(f[j.index, t.index] for t in aircraft.define_forbidden_positions_for_ULD(j)) == 0, name=f'C5_{j.index}')

    # DV7: Special ULD assignment - BAX/BUP/T ULDs assigned to exactly one position
    for j in cargo.uld:
        if j.isBAXorBUPorT:
            m.addConstr(quicksum(f[j.index, t.index] for t in aircraft.loadlocations) == 1, name=f'C_combi_2_{j.index}')
        # UNICA DIFERENCIA CON NUESTRO MODELO: 
        # BAX Position Fixed - BAX ULDs fixed to actual positions from real flight data 
        if j.isBAX:
            index_position_bax = [t.index for t in aircraft.loadlocations if t.location == j.actual_position_bax][0]
            m.addConstr(f[j.index, index_position_bax] == 1, name=f'C_BAX_fixed_{j.index}')

    # DV-Extra3: No items in special ULDs - Items cannot be loaded in BAX/BUP/T
    for i in cargo.items:
        for j in cargo.uld:
            if j.isBAXorBUPorT:
                m.addConstr(p[i.index, j.index] == 0, name=f'C_combi_3_{i.index}_{j.index}')

    M = max([i.weight for i in cargo.items])

    # DV8, DV9, DV10: Linking constraints - z_ijt active only if both p_ij and f_jt active
    for i in cargo.items:
        for j in cargo.uld:
            for t in aircraft.loadlocations:
                m.addConstr(z[i.index, j.index, t.index] <= p[i.index, j.index], name=f'C_lin_3_{i.index}_{j.index}_{t.index}')
                m.addConstr(z[i.index, j.index, t.index] <= f[j.index, t.index], name=f'C_lin_4_{i.index}_{j.index}_{t.index}')
                m.addConstr(z[i.index, j.index, t.index] >= p[i.index, j.index] + f[j.index, t.index] - 1, name=f'C_lin_5_{i.index}_{j.index}_{t.index}')

    # DV22: Overlapping positions - Overlapping positions cannot both be occupied
    for j_1 in cargo.uld:
        for j_2 in cargo.uld:
            if j_1 != j_2:
                for t_1 in aircraft.loadlocations:
                    for t_2 in aircraft.define_overlapping_positions(t_1):
                        m.addConstr(f[j_1.index, t_1.index] + f[j_2.index, t_2.index] <= 1, name=f'C6_{j_1.index}_{j_2.index}_{t_1.index}_{t_2.index}')

    # DV11: Position weight limit - Weight at each position cannot exceed limit, antes en codigo hacia dos restricciones weight de items <= M y weight uld<= M por separado. las combine como en el latex. The weight of a ULD assigned to a position should not exceed the maximum allowable weight. [Positions weight limit]
    for t in aircraft.loadlocations:
        m.addConstr(quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld if j.isBAXorBUPorT)
                <= aircraft.define_max_weight_postion(t), name=f'C7_{t.index}')
    
    # DV12: Compartment weight limits - Weight in each compartment (C1-C4) cannot exceed limit


    compartments = [
        ('C1', aircraft.max_weight_C1, aircraft.loadlocations_C1),
        ('C2', aircraft.max_weight_C2, aircraft.loadlocations_C2),
        ('C3', aircraft.max_weight_C3, aircraft.loadlocations_C3),
        ('C4', aircraft.max_weight_C4, aircraft.loadlocations_C4),
        # Add combined compartments if needed, e.g., ('C1_C2', aircraft.max_weight_C1_C2, aircraft.loadlocations_C1_C2)
    ]

    for compartment, max_weight, loadlocations in compartments:
        m.addConstr(
            quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in loadlocations) +
            quicksum(j.weight * f[j.index, t.index] for j in cargo.uld if j.isBAXorBUPorT for t in loadlocations)
            <= max_weight,
            name=f'C_Added_{compartment}'
        )

    # DV13: Forward compartment weight limit - Total weight in C1+C2 cannot exceed limit
    m.addConstr(
        quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1_C2) +
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld if j.isBAXorBUPorT for t in aircraft.loadlocations_C1_C2)
        <= aircraft.max_weight_C1_C2, name='C_Added_5'
    )

    # DV14: Aft compartment weight limit - Total weight in C3+C4 cannot exceed limit
    m.addConstr(
        quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3_C4) +
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld if j.isBAXorBUPorT for t in aircraft.loadlocations_C3_C4)
        <= aircraft.max_weight_C3_C4, name='C_Added_6'
     )
    
    # R33 The maximum takeoff weight (MTOW) limits TOW. The maximum landing weight (MLW) limits LW. The maximum zero fuel weight (MZFW) limits the ZFW
    m.addConstr(quicksum(i.weight * z[i.index, j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld for i in cargo.items) +
                quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld if j.isBAXorBUPorT)
                <= aircraft.define_MPL(), name=f'C8')

    # DV16: Lateral balance at landing - Left-right balance for LW (two constraints)
    # left - right
    m.addConstr((quicksum(i.weight * z[i.index, j_1.index, t_left.index] for i in cargo.items for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) +
                quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld if j_1.isBAXorBUPorT for t_left in aircraft.loadlocations_left)) - 
                (quicksum(i.weight * z[i.index, j_2.index, t_right.index] for i in cargo.items for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) +
                quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld if j_2.isBAXorBUPorT for t_right in aircraft.loadlocations_right)) <= 
                a_lat_TOW * (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations) + 
                            quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld if j.isBAXorBUPorT) + aircraft.OEW + aircraft.TOF) * b_lat_TOW, name=f'C9_1')
    #right -left
    m.addConstr((quicksum(i.weight * z[i.index, j_2.index, t_right.index] for i in cargo.items for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) +
                quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld if j_2.isBAXorBUPorT for t_right in aircraft.loadlocations_right)) - 
                (quicksum(i.weight * z[i.index, j_1.index, t_left.index] for i in cargo.items for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) +
                quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld if j_1.isBAXorBUPorT for t_left in aircraft.loadlocations_left)) <= 
                a_lat_TOW * (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations) +
                            quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld if j.isBAXorBUPorT) + aircraft.OEW + aircraft.TOF) * b_lat_TOW, name=f'C9_2')

    # DV15: Lateral balance at takeoff - Left-right balance for TOW (two constraints)
    m.addConstr((quicksum(i.weight * z[i.index, j_1.index, t_left.index] for i in cargo.items for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) +
                quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld if j_1.isBAXorBUPorT for t_left in aircraft.loadlocations_left)) - 
                (quicksum(i.weight * z[i.index, j_2.index, t_right.index] for i in cargo.items for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) +
                quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld if j_2.isBAXorBUPorT for t_right in aircraft.loadlocations_right)) <= 
                a_lat_LW * (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations) +
                            quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld if j.isBAXorBUPorT) + aircraft.OEW + aircraft.TOF - aircraft.TripF) * b_lat_LW, name=f'C10_1')
    # DV16 continued (second LW lateral constraint)
    m.addConstr((quicksum(i.weight * z[i.index, j_2.index, t_right.index] for i in cargo.items for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) +
                quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld if j_1.isBAXorBUPorT for t_right in aircraft.loadlocations_right)) - 
                (quicksum(i.weight * z[i.index, j_1.index, t_left.index] for i in cargo.items for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) +
                quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld if j_1.isBAXorBUPorT for t_left in aircraft.loadlocations_left)) <= 
                a_lat_LW * (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations) +
                            quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld if j.isBAXorBUPorT) + aircraft.OEW + aircraft.TOF - aircraft.TripF) * b_lat_LW, name=f'C10_2')


    #las 4 que siguen son la r26 yr27 del latex pero divididas por cargo load, en TOW y ZFW. pero en el latex tenemos para el conjunto E que tiene TOW, ZFW, y LW. 
    # LW no lo tiene, verificar si no lo quiere o se le paso.
    
    # DV17: Forward CG envelope limit - CG must be within forward limit for TOW
    m.addConstr(aircraft.define_INDEX_TOW_fwd(aircraft.aircraft_type) <= aircraft.DOI + aircraft.fuel_index + aircraft.define_INDEX_PAX() +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C1 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C2 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C3 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C4, name='C11_1')
    # 27 e = tow.
    m.addConstr(aircraft.DOI + aircraft.fuel_index + aircraft.define_INDEX_PAX() +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C1 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C2 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C3 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C4 <= 
                aircraft.define_INDEX_TOW_aft(aircraft.aircraft_type), name='C11_2')
    
    # DV18: Aft CG envelope limit - CG must be within aft limit for ZFW
    m.addConstr(aircraft.define_INDEX_ZFW_fwd(aircraft.aircraft_type) <= aircraft.DOI + aircraft.define_INDEX_PAX() +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C1 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C2 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C3 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C4, name='C12_1')
    # 27 e = zfw
    m.addConstr(aircraft.DOI + aircraft.define_INDEX_PAX() +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C1 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C2 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C3 +
                (quicksum(i.weight * z[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C4 <= 
                aircraft.define_INDEX_ZFW_aft(aircraft.aircraft_type), name='C12_2')


    # SPECIAL HANDLING

    COL_items_present = any(i.COL == 1 for i in cargo.items)
    CRT_items_present = any(i.CRT == 1 for i in cargo.items)
    COL_items_indices = [i.index for i in cargo.items if i.COL == 1]
    CRT_items_indices = [i.index for i in cargo.items if i.CRT == 1]
    T_with_COL = [j.index for j in cargo.uld if j.COL == 1]
    T_with_CRT = [j.index for j in cargo.uld if j.CRT == 1]

    # DV19: COL/CRT mixing prohibition - COL and CRT items cannot be in same ULD
    for j in cargo.uld:
        if j.isNeitherBAXnorBUPnorT:
            for i_1 in cargo.items:
                for i_2 in cargo.items:
                    if i_1 != i_2:
                        if i_1.COL == 1 and i_2.CRT == 1:
                            m.addConstr(p[i_1.index, j.index] + p[i_2.index, j.index] <= 1, name=f'C_special_1_{i_1.index}_{i_2.index}_{j.index}')

    
    # #R34 MODIFICAR A LA DEL PAPAEL DEL PROFE Y TAMBIEN EN EL LATEX 
    # DV24: COL/CRT compartment separation - COL and CRT cannot be in same compartment (Boeing 777)
    # if str(aircraft.aircraft_type) in ['772', '77W']:
    #     for k, loadlocations in [('front', aircraft.loadlocations_C1_C2), ('aft', aircraft.loadlocations_C3_C4)]:
    #         m.addConstr(
    #             quicksum(f[j.index, t.index] for j in cargo.uld if j.COL == 1 for t in loadlocations) +
    #             quicksum(f[j.index, t.index] for j in cargo.uld if j.CRT == 1 for t in loadlocations)
    #             <= 1, name=f'C_special_compartment_{k}'
    #         )
        
        
         # redundante con c4 y con R29

        # if COL_items_present and CRT_items_present:
        #     for j in cargo.uld:
        #         if j.isNeitherBAXnorBUPnorT:
        #             for i in COL_items_indices:
        #                 for k in CRT_items_indices:
        #                     for t in aircraft.loadlocations_C1_C2:
        #                         COL_C1_C2 = m.addVar(vtype=GRB.BINARY, name=f'COL_C1_C2_{i}_{j.index}_{t.index}')
        #                         CRT_C1_C2 = m.addVar(vtype=GRB.BINARY, name=f'CRT_C1_C2_{k}_{j.index}_{t.index}')
                                
        #                         m.addConstr(COL_C1_C2 <= p[i, j.index])
        #                         m.addConstr(COL_C1_C2 <= f[j.index, t.index])
        #                         m.addConstr(COL_C1_C2 >= p[i, j.index] + f[j.index, t.index] - 1)
                                
        #                         m.addConstr(CRT_C1_C2 <= p[k, j.index])
        #                         m.addConstr(CRT_C1_C2 <= f[j.index, t.index])
        #                         m.addConstr(CRT_C1_C2 >= p[k, j.index] + f[j.index, t.index] - 1)
        #                     m.addConstr(quicksum(COL_C1_C2 + CRT_C1_C2 for t in aircraft.loadlocations_C1_C2) == 0, name=f'C_special_COL_CRT_C1_C2_{i}_{k}_{j.index}')
       
        #                     for t in aircraft.loadlocations_C3_C4:
        #                         COL_C3_C4 = m.addVar(vtype=GRB.BINARY, name=f'COL_C3_C4_{i}_{j.index}_{t.index}')
        #                         CRT_C3_C4 = m.addVar(vtype=GRB.BINARY, name=f'CRT_C1_C2_{k}_{j.index}_{t.index}')
                                
        #                         m.addConstr(COL_C3_C4 <= p[i, j.index])
        #                         m.addConstr(COL_C3_C4 <= f[j.index, t.index])
        #                         m.addConstr(COL_C3_C4 >= p[i, j.index] + f[j.index, t.index] - 1)
                                
        #                         m.addConstr(CRT_C3_C4 <= p[k, j.index])
        #                         m.addConstr(CRT_C3_C4 <= f[j.index, t.index])
        #                         m.addConstr(CRT_C3_C4 >= p[k, j.index] + f[j.index, t.index] - 1)

        #                     m.addConstr(quicksum(COL_C3_C4 + CRT_C3_C4 for t in aircraft.loadlocations_C3_C4) == 0, name=f'C_special_COL_CRT_C3_C4_{i}_{k}_{j.index}')

    # revisar, estas no estan en el latex. 

    # DV25: COL/CRT rear prohibition - COL and CRT prohibited in aft (Boeing 787/781)
    # esta se deja pero arreglada 
    #PODRIAMOS PONERLO EN LA DEF DE VARIABLES Q SEAN 0 LOS FJT QUE TIENEN COL O CRT EN EL AFT COMPARTMEMT
    
    # if str(aircraft.aircraft_type) in ['789', '781']:
    #     for i in cargo.items:
    #         if i.COL == 1 or i.CRT == 1:
    #             for j in cargo.uld:
    #                 if j.isNeitherBAXnorBUPnorT:
    #                     for t in aircraft.loadlocations_C3_C4:
    #                         m.addConstr(z[i.index, j.index, t.index] == 0, name=f'C_special_COL_CRT_{i.index}_{j.index}_{t.index}')
    #     for t in aircraft.loadlocations_C3_C4:
    #         m.addConstr(
    #             quicksum(f[j.index, t.index] for j in cargo.uld if j.COL == 1) +
    #             quicksum(f[j.index, t.index] for j in cargo.uld if j.CRT == 1) == 0,
    #             name=f'C_special_T_{t.index}'
    #         )


    # DV24: COL/CRT compartment separation - COL and CRT cannot be in same compartment (Boeing 777)
    if str(aircraft.aircraft_type) in ['772', '77W']:
        compartments_R34 = {
            'front': aircraft.loadlocations_C1_C2,
            'aft': aircraft.loadlocations_C3_C4
        }
        
        COL_k = {}
        CRT_k = {}
        
        for k_label, loadlocations_k in compartments_R34.items():
            COL_k[k_label] = m.addVar(vtype=GRB.BINARY, name=f'COL_{k_label}')
            CRT_k[k_label] = m.addVar(vtype=GRB.BINARY, name=f'CRT_{k_label}')
            
            U_COL = [j for j in cargo.uld if j.COL == 1]
            U_CRT = [j for j in cargo.uld if j.CRT == 1]
            
            big_M_COL = len(U_COL) * len(loadlocations_k) if U_COL else 1
            big_M_CRT = len(U_CRT) * len(loadlocations_k) if U_CRT else 1
            
            if U_COL:
                m.addConstr(
                    quicksum(f[j.index, t.index] for j in U_COL for t in loadlocations_k) <= big_M_COL * COL_k[k_label],
                    name=f'C_R34_COL_{k_label}'
                )
            
            if U_CRT:
                m.addConstr(
                    quicksum(f[j.index, t.index] for j in U_CRT for t in loadlocations_k) <= big_M_CRT * CRT_k[k_label],
                    name=f'C_R34_CRT_{k_label}'
                )
            
            m.addConstr(
                COL_k[k_label] + CRT_k[k_label] <= 1,
                name=f'C_R34_COL_CRT_conflict_{k_label}'
            )
            
            for j in cargo.uld:
                if j.isNeitherBAXnorBUPnorT:
                    for i in cargo.items:
                        if i.COL == 1:
                            m.addConstr(
                                quicksum(z[i.index, j.index, t.index] for t in loadlocations_k) <= big_M_COL * COL_k[k_label],
                                name=f'C_R34_COL_item_{i.index}_{j.index}_{k_label}'
                            )
                        if i.CRT == 1:
                            m.addConstr(
                                quicksum(z[i.index, j.index, t.index] for t in loadlocations_k) <= big_M_CRT * CRT_k[k_label],
                                name=f'C_R34_CRT_item_{i.index}_{j.index}_{k_label}'
                            )

    # Special handling constraints for Boeing 787 (789, 781): No COL or CRT in aft, mutually exclusive in front
    if str(aircraft.aircraft_type) in ['789', '781']:
    # DV25a: Prohibit COL and CRT items/ULDs in aft compartment (C3+C4)
        for i in cargo.items:
            if i.COL == 1 or i.CRT == 1:
                for j in cargo.uld:
                    if j.isNeitherBAXnorBUPnorT:
                        for t in aircraft.loadlocations_C3_C4:
                            m.addConstr(
                                z[i.index, j.index, t.index] == 0,
                                name=f'C_787_COL_CRT_item_{i.index}_{j.index}_{t.index}'
                            )
        for t in aircraft.loadlocations_C3_C4:
            m.addConstr(
                quicksum(f[j.index, t.index] for j in cargo.uld if j.COL == 1 or j.CRT == 1) == 0,
                name=f'C_787_COL_CRT_uld_{t.index}'
            )
        
    # DV26: Mutually exclusive COL and CRT in front compartment (C1+C2)
        loadlocations_front = aircraft.loadlocations_C1_C2
        COL_front = m.addVar(vtype=GRB.BINARY, name='COL_front')
        CRT_front = m.addVar(vtype=GRB.BINARY, name='CRT_front')
        
        U_COL = [j for j in cargo.uld if j.COL == 1]
        U_CRT = [j for j in cargo.uld if j.CRT == 1]
        
        big_M_COL = len(U_COL) * len(loadlocations_front) if U_COL else 1
        big_M_CRT = len(U_CRT) * len(loadlocations_front) if U_CRT else 1
        
        if U_COL:
            m.addConstr(
                quicksum(f[j.index, t.index] for j in U_COL for t in loadlocations_front) <= big_M_COL * COL_front,
                name='C_787_COL_front'
            )
        
        if U_CRT:
            m.addConstr(
                quicksum(f[j.index, t.index] for j in U_CRT for t in loadlocations_front) <= big_M_CRT * CRT_front,
                name='C_787_CRT_front'
            )
        
        m.addConstr(
            COL_front + CRT_front <= 1,
            name='C_787_COL_CRT_conflict_front'
        )
        
        for j in cargo.uld:
            if j.isNeitherBAXnorBUPnorT:
                for i in cargo.items:
                    if i.COL == 1:
                        m.addConstr(
                            quicksum(z[i.index, j.index, t.index] for t in loadlocations_front) <= big_M_COL * COL_front,
                            name=f'C_787_COL_item_{i.index}_{j.index}_front'
                        )
                    if i.CRT == 1:
                        m.addConstr(
                            quicksum(z[i.index, j.index, t.index] for t in loadlocations_front) <= big_M_CRT * CRT_front,
                            name=f'C_787_CRT_item_{i.index}_{j.index}_front'
                        )
        #nuevas restricciones
    # DV-Note: LaTeX constraint r28 (redundant with DV2 - minimum items in active ULD)
    # DV-Note2: Minimum items constraint (LaTeX constraint, already covered by DV2)
        # redundante
        
        for j in cargo.uld:
            if j.isNeitherBAXnorBUPnorT:
                m.addConstr(
                    quicksum(p[i.index, j.index] for i in cargo.items) >= u[j.index],
                    name=f'C_min_items_{j.index}'
                )

    # # r30 y r31
    # DV20-Note: Booking separation upper bound (related to DV20)
    # for b_i, items in booking_groups.items():
    #     for j in cargo.uld:
    #         if j.isNeitherBAXnorBUPnorT:
    #             m.addConstr(
    #                 quicksum(p[i.index, j.index] for i in items) <= len(items) + M * (1 - SP[b_i, j.index]),
    # DV20-Note: Booking separation upper bound (related to DV20)
    #             )
    #             m.addConstr(
    #                 quicksum(p[i.index, j.index] for i in items) >= len(items) - M * SP[b_i, j.index],
    #                 name=f'C_separation_lower_{b_i}_{j.index}'
    #             )



    # New constraints 29
    # Constraint: P_ij <= Z_bi_j for each bi in serialnumbers, j in U, i in bi
    for b_i, items in booking_groups.items():
        for j in cargo.uld:
            if j.isNeitherBAXnorBUPnorT:
                for i in items:
                    m.addConstr(
                        p[i.index, j.index] <= Z[b_i, j.index],
                        name=f'C_P_Z_{b_i}_{i.index}_{j.index}'
                    )

    # DV21: Item separation (Y to Z) - Y counts number of ULDs used by booking
    for b_i in booking_groups.keys():
        m.addConstr(
            Y[b_i] == quicksum(Z[b_i, j.index] for j in cargo.uld if j.isNeitherBAXnorBUPnorT),
            name=f'C_Y_Z_{b_i}'
        )

              
    
    '''Optimize the model'''
    '''Optimize the model'''

    if not deferred_items_optimal_flag:
        envs = [m.getMultiobjEnv(i) for i in range(6)]
        envs[0].setParam(GRB.Param.TimeLimit, 60)  # %MAC
        #envs[1].setParam(GRB.Param.TimeLimit, 15)  # Volume preference
        envs[1].setParam(GRB.Param.TimeLimit, 15)  # Number of ULDs
        envs[2].setParam(GRB.Param.TimeLimit, 15)  # Total volume
        envs[3].setParam(GRB.Param.TimeLimit, 15)  # Separation
        envs[4].setParam(GRB.Param.TimeLimit, 15)  # BAX proximity
        m.optimize()
    else:
        m.setParam(GRB.Param.MIPGap, 0.1)
        envs = [m.getMultiobjEnv(i) for i in range(6)]
        envs[0].setParam(GRB.Param.TimeLimit, 60)
      #  envs[1].setParam(GRB.Param.TimeLimit, 15)
        envs[1].setParam(GRB.Param.TimeLimit, 15)
        envs[2].setParam(GRB.Param.TimeLimit, 15)
        envs[3].setParam(GRB.Param.TimeLimit, 15)
        envs[4].setParam(GRB.Param.TimeLimit, 15)
        m.optimize()


    status = m.status
    if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE: 
        print('The model is infeasible or unbounded')
        #m.computeIIS()
        m.write('model.ilp')
        print('Infeasibility report written to model.ilp')

    elif status == GRB.Status.TIME_LIMIT:
        print('Time limit reached')
    elif status == GRB.Status.OPTIMAL:
        print('Solution found')
    elif status == GRB.Status.INTERRUPTED:
        print('Optimization was stopped early')
    

    
    '''Results'''
    results_filename = 'Results.txt'
    folder_path = project_setup.setup_project_directory(aircraft.flight_number, aircraft.date, aircraft.departure_airport, aircraft.arrival_airport, baseline=False, optimized_actual=False, BAX_fixed=False)
    results_file_path = os.path.join(folder_path, results_filename)
    loadfactor_dict = {}

    with open(results_file_path, 'w') as file:
        total_weight = 0
        number_of_uld_solution = 0
        for t in aircraft.loadlocations:
            for j in cargo.uld:
                if f[j.index, t.index].x > 0.9999:
                    if j.isNeitherBAXnorBUPnorT:
                        number_of_uld_solution += 1
                        print(f'ULD {j.serialnumber} with weight {sum(i.weight * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999):.1f} kg with a volume loadfactor of {(sum(i.volume * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999) / j.volume) * 100:.3f}% and a weight loadfactor of {(sum(i.weight * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999) / j.max_weight) * 100:.3f}% is loaded to position {t.location}')
                        file.write(f'ULD {j.serialnumber} with weight {sum(i.weight * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999):.1f} kg with a volume loadfactor of {(sum(i.volume * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999) / j.volume) * 100:.3f}% and a weight loadfactor of {(sum(i.weight * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999) / j.max_weight) * 100:.3f}% is loaded to position {t.location}\n')
                        volume_loadfactor = (sum(i.volume * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999) / j.volume)
                        loadfactor_dict[j] = volume_loadfactor
                        total_weight += sum(i.weight * p[i.index, j.index].x for i in cargo.items)
                    if j.isBAXorBUPorT:
                        print(f'ULD {j.serialnumber} with weight {j.weight} kg is loaded to position {t.location}')
                        file.write(f'ULD {j.serialnumber} with weight {j.weight} kg is loaded to position {t.location}\n')
                        total_weight += j.weight
        # else:
        #     print(f'Model is infeasible')
        #     print('Status:', m.status)
        #     #m.computeIIS()
        #     # Handle infeasibility - maybe open new ULD or defer items

        print('---------------------------------------------------------------------------')
        file.write('---------------------------------------------------------------------------\n')
        print(f'Weight in Compartment 1: {sum(i.weight * z[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT):.1f} kg')
        file.write(f'Weight in Compartment 1: {sum(i.weight * z[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT):.1f} kg\n')
        print(f'Weight in Compartment 2: {sum(i.weight * z[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT):.1f} kg')
        file.write(f'Weight in Compartment 2: {sum(i.weight * z[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT):.1f} kg\n')
        print(f'Weight in Compartment 3: {sum(i.weight * z[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT):.1f} kg')
        file.write(f'Weight in Compartment 3: {sum(i.weight * z[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT):.1f} kg\n')
        print(f'Weight in Compartment 4: {sum(i.weight * z[i.index, j.index, t.index].x for i in cargo.items for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT):.1f} kg')
        file.write(f'Weight in Compartment 4: {sum(i.weight * z[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT):.1f} kg\n')

        actual_builded = [j for j in cargo.uld if not j.isBAXorBUPorT]

        # Calculating %MAC
        TOW_index = aircraft.DOI + aircraft.fuel_index + aircraft.define_INDEX_PAX()
        ZFW_index = aircraft.DOI + aircraft.define_INDEX_PAX()

        dict_loadlocations = {aircraft.delta_index_cargo_C1: aircraft.loadlocations_C1, aircraft.delta_index_cargo_C2: aircraft.loadlocations_C2, aircraft.delta_index_cargo_C3: aircraft.loadlocations_C3, aircraft.delta_index_cargo_C4: aircraft.loadlocations_C4}

        for i in cargo.items:
            for j in cargo.uld:
                for value, compartment in dict_loadlocations.items():
                    for t in compartment:
                        if f[j.index, t.index].x > 0.9999 and j.isNeitherBAXnorBUPnorT:
                            TOW_index += i.weight * z[i.index, j.index, t.index].x * float(value)
                            ZFW_index += i.weight * z[i.index, j.index, t.index].x * float(value)

        for j in cargo.uld:
            for value, compartment in dict_loadlocations.items():
                for t in compartment:
                    if f[j.index, t.index].x > 0.9999 and j.isBAXorBUPorT:
                        TOW_index += j.weight * f[j.index, t.index].x * float(value) 
                        ZFW_index += j.weight * f[j.index, t.index].x * float(value)

        increment_value = aircraft.define_ff_increment_MAC_ZFW(MAC_obj.getValue())
        fuel_saving_kg = aircraft.TripF * (increment_value / 100)

        print('===========================================================================')
        file.write('===========================================================================\n')
        print(f'%MAC ZFW is {MAC_obj.getValue()}')
        file.write(f'%MAC ZFW is {MAC_obj.getValue()}\n')
        file.write(f'%Number of ULDs used is {obj2.getValue()}\n')
        file.write(f'%Total Volume Packed is {obj_volume_total.getValue()}\n')
        file.write(f'%Booking Separation Penalty is {obj_separation.getValue()}\n')
        file.write(f'%BAX Proximity Score is {obj4.getValue()}\n')
        print('---------------------------------------------------------------------------')
        file.write('---------------------------------------------------------------------------\n')
        print(f'The actual %MAC ZFW for this flight was {aircraft.actual_MAC_ZFW}')
        file.write(f'The actual %MAC ZFW for this flight was {aircraft.actual_MAC_ZFW}\n')
        print(f'Resulting in a fuel deviation of {increment_value:.3f}% or {fuel_saving_kg:.3f} kg')
        file.write(f'Resulting in a fuel deviation of {increment_value:.3f}% or {fuel_saving_kg:.3f} kg\n')
        print('---------------------------------------------------------------------------')
        file.write('---------------------------------------------------------------------------\n')
        print(f'{number_of_uld_solution} ULDs are built by the model')
        file.write(f'{number_of_uld_solution} ULDs are built by the model\n')
        print(f'{cargo.total_number_of_build_ULDs} ULDs were actually built')
        file.write(f'{cargo.total_number_of_build_ULDs} ULDs were actually built\n')
        print('---------------------------------------------------------------------------')
        print(f'The ULDs are built and blocked in the model:')
        for uld, items in packed_ULDs.items():
            print(f'{str(uld)}')

    # Making results dictionaries
    results_1D_BPP_WB = {}

    for j in cargo.uld:
        if u[j.index].x == 1:
            if j.isNeitherBAXnorBUPnorT:
                results_1D_BPP_WB[j] = []
            # puetso por mi
            # volume = sum(i.volume * p.get((i.index, j.index), 0) for i in cargo.items)
            # print(f"ULD {j.serialnumber}: Volumen {volume}, Capacidad {j.volume}, Loadfactor {volume/j.volume*100:.2f}%")
            # items = [i for i in cargo.items if p.get((i.index, j.index), 0) > 0.999]
            # has_COL = any(i.COL == 1 for i in items)
            # has_CRT = any(i.CRT == 1 for i in items)
            # print(f"ULD {j.serialnumber}: COL={has_COL}, CRT={has_CRT}")
            # assigned_positions = [t.serialnumber for t in aircraft.loadlocations if f[j.index, t.index].x > 0.999]
            # print(f"ULD {j.serialnumber}: Posiciones asignadas {assigned_positions}")

    for i in cargo.items:
        for j in cargo.uld:
            if p[i.index, j.index].x > 0.999:
                results_1D_BPP_WB[j].append(i)

    number_of_items_in_results = 0
    for j, items in results_1D_BPP_WB.items():
        number_of_items_in_results += len(items)
    print('---------------------------------------------------------------------------')
    
    return results_1D_BPP_WB, ZFW_index, TOW_index, loadfactor_dict

In [48]:
def solve_3D_BPP(results_1D_BPP_WB):
    deferred_items = {}
    all_placed_items = {}
    all_extreme_points = {}

    for j, items in results_1D_BPP_WB.items():
        extreme_points = EP.get_starting_extreme_points(j)
        items_to_place = items.copy()
        placed_items = {}
        deferred_items[j] = []

        while items_to_place:
            next_item, placement_details, defer_reason = EP.find_best_next_item_and_placement(items_to_place, j, placed_items, extreme_points)

            if next_item:
                best_ep, best_orientation, best_merit, best_support_count = placement_details
                placed_items, added_points, removed_points = EP.place_item(next_item, best_ep, placed_items, best_orientation, extreme_points)
                extreme_points = EP.update_extreme_points(extreme_points, placed_items[next_item.serialnumber], next_item, j)
                items_to_place.remove(next_item)

            else:
                print(f'Items deferred for ULD {j.serialnumber}:')
                for item in items_to_place:
                    print(f'{item.serialnumber} deferred due to {defer_reason}')
                    deferred_items[j].append(item)
                print('---------------------------------------------------------------------------')

                items_to_place = []

        all_placed_items[j] = placed_items
        all_extreme_points[j] = extreme_points

    return deferred_items, all_placed_items, all_extreme_points


In [ ]:
def feedback_loop(cargo):
    start_time = time.time()
    total_time_1D_BPP_WB = 0
    total_time_3D_BPP = 0
    iteration = 1
    number_of_opened_uld = 0
    open_new_uld = False 
    open_new_extra_uld = False
    extra_uld_opened_in_previous_iteration = False
    solution_found = False
    deferred_items = {}
    previous_deferred_items = {}
    packed_ULDs = {}
    attempted_combinations = set()

    number_of_uld_actually_used = len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT])
    color_map = EP.get_color_map(cargo.items)
    ## aca tambien lo modifique y le puse bax = false, le faltaba
    folder_path = project_setup.setup_project_directory(aircraft.flight_number, aircraft.date, aircraft.departure_airport, aircraft.arrival_airport, baseline = False, optimized_actual = False, BAX_fixed = False)


    while not solution_found:
        print(f'Iteration {iteration}')
        print('===========================================================================')
        print(f'Starting optimization for iteration {iteration}')
        start_time_1D_BPP_WB = time.time()
        results_1D_BPP_WB_optimal, ZFW_index_optimal, TOW_index_optimal, loadfactor_dict_optimal = solve_1D_BPP_WB(cargo, deferred_items = deferred_items, attempted_combinations = attempted_combinations, 
                                                                                                                   number_of_opened_uld = number_of_opened_uld, packed_ULDs = packed_ULDs, 
                                                                                                                   open_new_uld = open_new_uld, open_new_extra_uld = open_new_extra_uld, 
                                                                                                                   deferred_items_optimal_flag = False)
        open_new_extra_uld = False
        total_time_1D_BPP_WB += time.time() - start_time_1D_BPP_WB
        number_of_opened_uld = sum(1 for value in results_1D_BPP_WB_optimal.values() if value)

        start_time_3D_BPP = time.time()
        deferred_items_optimal, placed_items_optimal, extreme_points_optimal = solve_3D_BPP(results_1D_BPP_WB_optimal)
        total_time_3D_BPP += time.time() - start_time_3D_BPP

        for j, items in deferred_items_optimal.items():
            if ((len(items) == 0) and (loadfactor_dict_optimal[j] >= 0.65)) or ((loadfactor_dict_optimal[j] >= 0.75) and (len(deferred_items_optimal[j]) <= 2)):
                if len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT]) - len(packed_ULDs) > 2:
                    packed_ULDs[j.serialnumber] = list(placed_items_optimal[j].keys())

        number_of_deferred_items_optimal = sum(len(items) for items in deferred_items_optimal.values())

        if number_of_deferred_items_optimal == 0:
            solution_found = True
            print('Optimal solution found with no deferred items')
            print('===========================================================================')
            plot.WB(aircraft, ZFW_index_optimal, TOW_index_optimal, folder_path)
            plot.BPP(cargo, results_1D_BPP_WB_optimal, placed_items_optimal, extreme_points_optimal, color_map, folder_path)
            end_time = time.time()
            break

        deferred_items = deferred_items_optimal
        current_iteration_deferred_flag = False
        
        for j, items in deferred_items_optimal.items():
            for i in items:
                previous_deferred_items[i.serialnumber] = previous_deferred_items.get(i.serialnumber, 0) + 1

                if ((previous_deferred_items[i.serialnumber] >= (number_of_opened_uld - len(packed_ULDs.keys()))) or 
                    ((number_of_opened_uld < len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT])) and (sum(1 for value in deferred_items_optimal.values() if value) == 1)) and
                    not extra_uld_opened_in_previous_iteration):

                    print(f'Flag set: opening a new ULD')
                    print(f'Item {i.serialnumber} is causing the flag')
                    print('---------------------------------------------------------------------------')
                    current_iteration_deferred_flag = True
                    if number_of_opened_uld < number_of_uld_actually_used:
                        number_of_opened_uld = number_of_opened_uld + 1
                        previous_deferred_items = {k: 0 for k in previous_deferred_items.keys()} 
                        extra_uld_opened_in_previous_iteration = False
                    elif number_of_opened_uld >= number_of_uld_actually_used:
                        open_new_extra_uld = True
                        number_of_opened_uld = number_of_opened_uld + 1
                        print(number_of_opened_uld)
                        previous_deferred_items = {k: 0 for k in previous_deferred_items.keys()}
                        extra_uld_opened_in_previous_iteration = True
                    break

        print(f'{number_of_deferred_items_optimal} items have been deferred this iteration')
        print(f'{sum(1 for value in deferred_items_optimal.values() if value)} ULDs have deferred items this iteration')

        if not current_iteration_deferred_flag and (sum(1 for value in deferred_items_optimal.values() if value) > 1):
            print(f'Continuing with optimal strategy due to high number of deferred items')
            print('===========================================================================')
            open_new_uld = False

        if not current_iteration_deferred_flag and (sum(1 for value in deferred_items_optimal.values() if value) == 1):
            print(f'Continuing with suboptimal startegy')
            print('===========================================================================')
            start_time_1D_BPP_WB_suboptimal = time.time()
            results_1D_BPP_WB_suboptimal, ZFW_index_suboptimal, TOW_index_suboptimal, loadfactor_dict_suboptimal = solve_1D_BPP_WB(cargo, deferred_items = deferred_items, attempted_combinations = attempted_combinations, 
                                                                                                                                   number_of_opened_uld = number_of_opened_uld, packed_ULDs = packed_ULDs, open_new_uld = open_new_uld, 
                                                                                                                                   open_new_extra_uld = open_new_extra_uld, deferred_items_optimal_flag = True)
            total_time_1D_BPP_WB += time.time() - start_time_1D_BPP_WB_suboptimal


            start_time_3D_BPP_suboptimal = time.time()
            deferred_items_suboptimal, placed_items_suboptimal, extreme_points_suboptimal = solve_3D_BPP(results_1D_BPP_WB_suboptimal)
            total_time_3D_BPP += time.time() - start_time_3D_BPP_suboptimal

            for j, items in deferred_items_suboptimal.items():
                if ((len(items) == 0) and (loadfactor_dict_suboptimal[j] >= 0.65)) or ((loadfactor_dict_suboptimal[j] >= 0.75) and (len(deferred_items_suboptimal[j]) <= 2)):
                    if len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT]) - len(packed_ULDs) > 2:
                        packed_ULDs[j.serialnumber] = list(placed_items_suboptimal[j].keys())

            number_of_deferred_items_suboptimal = sum(len(items) for items in deferred_items_suboptimal.values())
            
            if number_of_deferred_items_suboptimal == 0:
                solution_found = True
                print('Suboptimal solution found with no deferred items')
                print('===========================================================================')
                plot.WB(aircraft, ZFW_index_suboptimal, TOW_index_suboptimal, folder_path)
                plot.BPP(cargo, results_1D_BPP_WB_suboptimal, placed_items_suboptimal, extreme_points_suboptimal, color_map, folder_path)
                end_time = time.time()
                break

            for j, items in deferred_items_suboptimal.items():
                combination = (j, tuple(results_1D_BPP_WB_suboptimal[j]))
                attempted_combinations.add(combination)

            deferred_items = deferred_items_suboptimal

            print(f'{number_of_deferred_items_suboptimal} items have been deferred this suboptimal optimization')
            print('===========================================================================')

        iteration += 1

    model_info_path = os.path.join(folder_path, 'Model_Information.txt')

    with open(model_info_path, 'w') as file:
        file.write(f'Number of Iteration: {iteration}\n')
        file.write('---------------------------------------------------------------------------\n')
        file.write(f'Total time: {end_time - start_time:.3f} seconds\n')
        file.write('---------------------------------------------------------------------------\n')
        file.write(f'Total time 1D BPP WB: {total_time_1D_BPP_WB:.3f} seconds\n')
        file.write('---------------------------------------------------------------------------\n')
        file.write(f'Total time 3D BPP: {total_time_3D_BPP:.3f} seconds\n')

    return 

In [ ]:
feedback_loop(cargo)